In [1]:
cd $LIBRISPEECH
pwd

/projects/speech/ASR/kaldi/egs/librispeech/s5-mats


In [2]:
cat script/wav.sh

mkdir -p data/train21/build
ls /projects/speech/cl21/data/*/*_a/wav/*.wav > data/train21/build/wavname_a.txt
ls /projects/speech/cl21/data/*/*_b/wav/*.wav > data/train21/build/wavname_b.txt
ls /projects/speech/cl21/data/*/*_c/wav/*.wav > data/train21/build/wavname_c.txt
# All files pass this.
# for f in `cat data/train21/build/wavname_a.txt`; do file $f; done | egrep -v 'mono 16000'
cat data/train21/build/wavname_?.txt | cut -f 6,7 -d'/' | sort -u | tr '/' ' ' > data/train21/build/wavpart1.txt
# Find items where text, truth, and wav exist
cat data/train21/build/wavpart1.txt  | awk -f script/exists.awk > data/train21/build/wavpart2.sh
source data/train21/build/wavpart2.sh > data/train21/build/wavpart2.txt
# Remove known bad pairs
# Concatenate the text files
cat data/train21/build/wavpart2.txt | awk 'BEGIN {FS=" "}NF==2{printf("cat /projects/speech/cl21/data/%s/%s/text\n",$1,$2)}' > data/train21/build/text.sh
source data/train21/build/text.sh | sort -u | egrep '[A-Z]' > data/train21/bui

In [3]:
mkdir -p data/train21b 
cp data/train21/build/text data/train21b 
head -4 data/train21b/text

aiden-a-1 AT LEAST ONE VOWEL IS CAPITALIZED
aiden-a-10 THERE IS ONLY ONE T
aiden-a-2 SOME VOWEL IS NOT CAPITALIZED
aiden-a-3 EVERY GLIDE PRECEDES A VOWEL


In [4]:
cat data/train21b/text | cut -f1 -d' ' | awk 'BEGIN {FS="-"}{printf("%s %s\n",$0,$1)}' > data/train21b/utt2spk
head -4 data/train21b/utt2spk

aiden-a-1 aiden
aiden-a-10 aiden
aiden-a-2 aiden
aiden-a-3 aiden


In [16]:
cp data/train21/build/truth data/train21b 
head -4 data/train21b/truth

aiden-a-1 aTqF f Attq t Atta t AAAA t tstt u ieou f euEu t THHH u
aiden-a-2 aTqF t Attq f Atta t AAAA f tstt u ieou t euEu t THHH u
aiden-a-3 agqF u Atya t yayt f AAAA u yaYi t yyyy f euEy f aYaa t
aiden-a-4 aT u Attqa f AttaT t AAAAt t tstta f ieOuli t edhGHth f THHHa f


In [8]:
utils/utt2spk_to_spk2utt.pl data/train21b/utt2spk > data/train21b/spk2utt
tail -2 data/train21b/spk2utt
ls data/train21b

olivia olivia-a-1 olivia-a-10 olivia-a-2 olivia-a-3 olivia-a-4 olivia-a-5 olivia-a-6 olivia-a-7 olivia-a-8 olivia-a-9 olivia-b-1 olivia-b-2 olivia-b-3 olivia-b-4 olivia-b-5 olivia-c-1 olivia-c-2 olivia-c-3 olivia-c-4 olivia-c-5
zoe zoe-a-1 zoe-a-10 zoe-a-2 zoe-a-3 zoe-a-4 zoe-a-5 zoe-a-6 zoe-a-7 zoe-a-8 zoe-a-9 zoe-b-1 zoe-b-2 zoe-b-3 zoe-b-4 zoe-b-5 zoe-c-1 zoe-c-2 zoe-c-3 zoe-c-4 zoe-c-5
spk2utt  text  utt2spk


In [10]:
cat data/train21b/text | awk 'BEGIN {FS=" "}{split($1,p,"-"); printf("%s /projects/speech/cl21/data/%s/%s_%s/wav/%s.wav\n",$1,p[1],p[1],p[2],$1)}' > data/train21b/wav.scp
head -2 data/train21b/wav.scp

aiden-a-1 /projects/speech/cl21/data/aiden/aiden_a/wav/aiden-a-1.wav
aiden-a-10 /projects/speech/cl21/data/aiden/aiden_a/wav/aiden-a-10.wav


In [12]:
# rm data/train21b/conf/mfcc.conf
steps/make_mfcc.sh --cmd run.pl --nj 1 data/train21b mfcc

steps/make_mfcc.sh --cmd run.pl --nj 1 data/train21b mfcc
steps/make_mfcc.sh: moving data/train21b/feats.scp to data/train21b/.backup
utils/validate_data_dir.sh: Successfully validated data-directory data/train21b
steps/make_mfcc.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
steps/make_mfcc.sh: Succeeded creating MFCC features for train21b


In [14]:
steps/compute_cmvn_stats.sh data/train21b mfcc
utils/validate_data_dir.sh data/train21b

steps/compute_cmvn_stats.sh data/train21b mfcc
Succeeded creating CMVN stats for train21b
utils/validate_data_dir.sh: Successfully validated data-directory data/train21b


In [9]:
# Decode as a check on train21b
steps/decode_fmllr.sh --nj 1 --cmd run.pl exp/tri6b/graph_tgsmall data/train21b exp/tri6b/decode_tgsmall_train21B

In [17]:
egrep '^zoe-a' exp/tri6b/decode_tgsmall_train21B.si/log/decode.1.log | head 

zoe-a-1 THERE IS MORE THAN ONE CAPITALIZED VOWEL 
zoe-a-10 THERE ARE NO CAPITALIZED LETTERS 
zoe-a-2 A CONSONANT IMMEDIATELY PRECEDES EVERY VOWEL 
zoe-a-3 LETTER TO IS AT THE CAPITAL OR LETTER THREE IS UNIQUE 
zoe-a-4 ALL CAPITALIZED VOWELS ARE FINAL 
zoe-a-5 AT LEAST ONE GLIDE IS REPEATED 
zoe-a-6 THERE ARE VOICED CONSONANTS 
zoe-a-7 ALL VOWELS ARE CAPITALIZED AND ALL CONSONANTS ARE NOT CAPITALIZED 
zoe-a-8 ONLY GLIDES ARE REPEATED 
zoe-a-9 A VOWEL IS ADJACENT TO EVERY CONSONANT 


#### Best sentences
Some of this modified from Elise Kronblicher's 2021 project report.

Read into a list the results of `lattice-to-nbest` and `nbest-to-linear`, with an $n$ of 100. See the earlier notebook.

In [11]:
best_100_path = exp + 'exp/tri6b/decode_tgsmall_train21d.si/100.text'
with open(best_100_path) as f:
    best_100_lines=f.readlines()
f.close()

There are a lot, because there are up to 100 readings for each audio.

In [14]:
len(best_100_lines)

19903

The following is slow, it took 9m 30.6s.

In [13]:
idx=[]
for s in best_100_lines:
    id='-'.join(s.split('-')[0:3])
    split=s.split()
    idx+=[[id,split[0],[s.lower() for s in split[1:]]]]

files=[]
sents=[]
for i in idx:
    if i[0] not in files:
        if sentence_parses(i[2],p1):
            files+=[i[0]]
            sents+=[' '.join(w.upper() for w in i[2])]
for i in idx:
    if i[0] not in files:
        files+=[i[0]]
        sents+=[' '.join(w.upper() for w in i[2])]
print(files[0:3],sents[0:3])


['aiden-a-1', 'aiden-a-2', 'aiden-a-3'] ['AT LEAST ONE VOWEL IS CAPITALIZED', 'SOME VOWEL IS NOT CAPITALIZED', 'EVERY GLIDE PRECEDES THE VOWEL']


In [15]:

# This saves the results of the slow code above.  It has the best reading for each uid.
# It's suboptimal, it mixes in sentences that did not parse.
f = open(s5 + '/result/result1', "w")
for i in range(len(files)):
    f.write(files[i]+" "+sents[i]+"\n")
f.close()



## Code for models

In [27]:
import nltk
from nltk import grammar, parse, load_parser
from typing import Callable, List, Set
import re

###This code from Akash Aryal's model generator 

vowels = ['a', 'e', 'i', 'o', 'u']
fricatives = ['v', 'f', 's', 'z', 'h', 'th', 'sh', 'zh']
              
#def vowel(word,i):
#    return word[i].lower() in vowels
#get_vowel = lambda word: f'vowel => {set([i+1 for i in range(len(word)) if vowel(word,i)])}'

def capital(word, i):
    return word[i].isupper()
get_capital = lambda word: f'capital => {set([i+1 for i in range(len(word)) if capital(word,i)])}'

def less_than(i, j):
    return i<j
get_less_than = lambda word: f'le => {set([(i+1,j+1) for i in range(len(word)) for j in range(len(word)) if i!=j and less_than(i+1,j+1)])}'

def adjacent(i,j):
    return abs(i-j) == 1
get_adjacent = lambda word: f'ad => {set([(i+1,j+1) for i in range(len(word)) for j in range(len(word)) if adjacent(i+1,j+1)])}'

get_even =  lambda word: f'even => {set([i+1 for i in range(len(word)) if (i+1)%2==0])}'

get_odd = lambda word: f'odd => {set([i+1 for i in range(len(word)) if (i+1)%2!=0])}'

# @323
def voiced(word):
    word=word.lower()
    v=[]
    for i in range(len(word)):
        if i != len(word)-1:
            if (word[i] == 'n' and word[i+1]=='g') or (word[i] == 's' and word[i+1] == 'z'):
                v.append((i+1, word[i]))
                v.append((i+2, word[i+1]))
        if word[i] == 'z':
            if ((i,'s') not in v):
                v.append((i+1, 'z'))
        if word[i] == 'g':
            if ((i,'n') not in v):
                v.append((i+1, 'g'))
        if word[i] == 'n':
            if ((i+2,'g') not in v):
                v.append((i+1,'n'))
        if word[i] in ['a', 'e', 'i', 'o', 'u', 'b', 'd', 'j', 'l', 'm', 'r', 'v', 'w', 'y']:
            v.append((i+1, word[i]))
    return v
            
get_voiced= lambda w: f'voiced => {set([(i+1,w[i].lower()) for i in range(len(w)) if (i+1, w[i].lower()) in voiced(w)])}'

def centered(word,i):
    if (len(word)%2==0):
        return len(word)//2 == i or i == len(word)//2 + 1
    else:
        return len(word)//2 + 1 == i

get_centered = lambda word: f'cent => {set([i+1 for i in range(len(word)) if centered(word,i+1)])}'

get_mirrored = lambda w: f'mirrored => {set(i+1 for i in range(len(w)) if w[i].lower() == w[len(w)-1-i].lower())}'

get_glide =  lambda w: f'glide => {set(i+1 for i in range(len(w)) if w[i].lower() == "w" or w[i].lower() == "y")}'

# from @325
def fricatives(word):
    word.lower()
    frics = []
    for i in range(len(word)):
        if i != len(word)-1:
            if word[i] in ['t', 's', 'z'] and word[i+1] == 'h':
                frics.append((i+1, word[i]))
                frics.append((i+2, 'h'))
        if word[i] == 'h':
            if ((i, 't') not in frics) and ((i, 's') not in frics) and ((i, 'z') not in frics):
                frics.append((i+1, 'h'))
        if word[i] in ['s', 'z']:
            if ((i+2, 'h') not in frics):
                frics.append((i+1, word[i]))
        if word[i] in ['v', 'f']:
            frics.append((i+1, word[i]))
        return frics
get_fricative = lambda w: f'fricative => {set([(i+1,w[i].lower()) for i in range(len(w)) if (i+1, w[i].lower()) in fricatives(w)])}'

##Code by Angela Liu from HW3
def to_model_str(word: str, special_rels: List[Callable[[str], str]]=[]) -> str:
    """
    Creates the string form of the model for the input word. This string is meant to be passed to `nltk.Valuation.fromstring`.
    By default, the function will only add the relations mapping i => i for i from 1 to the length of `word` and a relation 
    mapping char => the set of tuples (i, word[i]). The `special_rels` function allows you to specify additional relations to 
    be added to the valuation string.
    
    :param word: The word to create a model string for.
    :param special_rels: A list of functions that when called return a string of the form {relation_name} => {relation_contents}. Defaults to the empty list.
    :returns: a string representing the model for word
    """
    n = len(word)
    model_str = []
    char = []
    for i in range(1, n+1):
        model_str.append(f'{i} => {i}')
        char.append((i, word[i-1]))
    model_str.append(f'char => {set(char)}'.lower())
    return '\n'.join(model_str + [rel(word) for rel in special_rels]).replace("'", "")
    # Angela Liu

def emptysets(val:nltk.sem.evaluate.Valuation):
    val.update([(k,set()) for (k,v) in val.items() if v == 'set()'])
    



In [30]:
get_capital('aDvOcAtEf')

'capital => {8, 2, 4, 6}'

### Truth checking
From E.K.

In [27]:
truth_path = train21 + '/truth'
result_path = s5 + '/result/result1'
print(truth_path)
print(result_path)

/projects/speech/ASR/kaldi/egs/librispeech/s5-mats/data/train21/truth
/projects/speech/ASR/kaldi/egs/librispeech/s5-mats/result/result1


In [38]:
def check_truth(truthpath,resultpath,grammarpath):
    """
    Compares the truth values from user's generated tests to those produced from from the truth page. Iterates through the truth file
    one sentence at a time
     
    
    :param: Results are in the same format as a text file"
    """
    
    ##User needs to provide their own truth and text paths
    
    gram_lst=[]
    after_assign=[]
    after_calc=[]
    before_calc=[]
    text = open(resultpath)
    truth = open(truthpath)
    sentences = []
    truthTests = []
    for l, t in zip(text, truth):
        truthTests.append(t.strip('\n').split(" "))
        sentences.append(l.split(" ", 1)[0])
        sentences.append(l.split(" ", 1)[1].strip('\n').strip())
    
    pr = nltk.load_parser(grammarpath, trace=0,cache=False)
    #Magic, do not touch! Cannot parse sentences without this line!
    get_vowel = lambda w: f'vowel => {set(re.findall(r"[AEIOUaeiou]", w))}'.lower()
    
    numCorrect = 0 
    numGuessedCorrect=0
    numTests = 0
    for t in truthTests:
        words = []
        truths = []
        sentence = sentences[sentences.index(t[0])+1].lower()
        print(sentence)
        
        for i in range(1, len(t)):
                if (i % 2 == 1):
                    words.append(t[i])
                    
                else :
                    if(t[i] == 't'): 
                        truths.append('True')
                        numTests+=1
                    elif(t[i] == 'f'): 
                        truths.append('False')
                        numTests+=1
                    elif(t[i] == 'u'): truths.append('Undefined')
                    else: print("Check the format of your truth file, "+t[i])
        try:
            formula = next(pr.parse(sentence.split())).label()['SEM']
            print(formula)
            gram_lst+=[sentence]
               
            equals = lambda w: f'eq => {set([(i+1,i+1) for i in range(len(w))])}'
            vals = [nltk.Valuation.fromstring(to_model_str(w, [get_vowel, get_capital, get_less_than, get_adjacent, get_even, get_odd, 
                                                   get_voiced, get_centered, get_mirrored, get_glide, get_fricative])) for w in words]
            assignments = [nltk.Assignment(val.domain) for val in vals]
            for val in vals: emptysets(val)
            models = [nltk.Model(val.domain, val) for val in vals]
            after_assign+=[sentence]
            for w, a, m, t in zip(words, assignments, models, truths):
                calculated = m.evaluate(str(formula),a)
                
                #I left in some useful print statements if you want to see a breakdown of the results
                print(f'{w}\nexpected: {t} calculated: {calculated}\n----------------\n')
                if(t != "Undefined"):
                    print(f"truth file: {t} calculated: {calculated}")
                    if(t == str(calculated)): numCorrect+=1
                
            
        except:
            for t in truths:
                if t=='False': numGuessedCorrect+=1
#             print("\nYour sentence '" + sentence + "' is ungrammatical")
            
        
    print(f"\n{numCorrect+numGuessedCorrect} out of {numTests}\nTruth value error rate: {1-(numCorrect+numGuessedCorrect)/numTests}")
    print(numCorrect)
    print(numGuessedCorrect)
    print(numTests)
    

In [39]:
check_truth(truth_path,result_path,g1_path)

there is exactly one the t
at least one vowel is capitalized
exists n.(exists c.(vowel(c) & char(n,c)) & exists c.(capital(n) & char(n,c)))
some vowel is not capitalized
exists n.(exists c.(vowel(c) & char(n,c)) & -exists c.(capital(n) & char(n,c)))
every glide precedes the vowel
all n.(exists c.(glide(c) & char(n,c)) -> exists z7383.(exists c.(vowel(c) & char(z7383,c)) & le(n,z7383)))
the letter or that precedes that letter five is a vowel
letter three is unique
all c m.((char(m,c) & char(3,c)) -> (m = 3))
the penultimate letter is not a vowel
exists n.(exists c.char(n,c) & exists d.(ad(n,d) & le(n,d) & -exists m.le(d,m)) & exists c.char(n,c) & exists c.(vowel(c) & char(n,c)))
no consonant is unique
all n.(exists c.(-vowel(c) & char(n,c)) -> -all c m.((char(m,c) & char(n,c)) -> (m = n)))
there is at least one glide
exists n.(exists c.(glide(c) & char(n,c)) & exists c.char(n,c))
only letter two is unique
exists h.all m.(char(2,h) & (-(m = 2) -> (all c m.((char(m,c) & char(2,c)) -> (m =

In [30]:
print(numCorrect)
print(numGuessedCorrect)
print(numTests)

NameError: name 'numCorrect' is not defined